In [1]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
df_train = pd.read_json("icecat_preprocessed_train_keybert.json")
df_train.head()

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text,cleaned_text,keywords
0,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,<b>Smart Multimedia Performance</b><br>\nVivoP...,ASUS K31CD-IT049T. Processor frequency: 3.4 GH...,"ASUS K31CD-IT049T, 3.4 GHz, 6th gen Intel® Cor...","Intel Core i7-6700 (8M Cache, 3.4GHz), 16GB RA...",Computers & Electronics>Computers>PCs/Workstat...,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,asus pc gen gb gb hdd black tower b smart mult...,"[hardware, audio, vivopc, asus, streaming, ram..."
1,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,Get the performance you demand at a price that...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,1m ST/SC Plenum-Rated 9/125 Duplex Single-Mode...,Computers & Electronics>Computer Cables>Fibre ...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,duplex fiber patch cable blue fiber optic cabl...,"[ethernet, fiber, cable, gigabit, duplex, tran..."
2,HP FA889AA Battery,Keeping an extra source of power nearby means ...,"HP FA889AA. Product type: Battery, Product col...","HP FA889AA, Battery, White, Lithium-Ion (Li-Io...","1100 mAh, Lithium Ion, Standard Battery",Computers & Electronics>Computers>Handheld Mob...,HP FA889AA Battery Keeping an extra source of ...,hp battery keeping extra source power nearby m...,"[hp, battery, power, ipaq, office, standard, l..."
3,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,The C30 builds on its award-winning design as ...,Lenovo ThinkStation C30. Processor frequency: ...,"Lenovo ThinkStation C30, 2 GHz, Intel® Xeon® E...","Intel Xeon E5-2620 (15M Cache, 2.00 GHz, 7.20 ...",Computers & Electronics>Computers>PCs/Workstat...,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,lenovo thinkstation family gb gb hdd black tow...,"[lenovo, hardware, workstation, intel, process..."
4,Lenovo 00D8199 software license/upgrade,SUSE Linux Enterprise Server (SLES)<br>\n<br>\...,Lenovo 00D8199. License term in years: 3 year(s),"Lenovo 00D8199, 3 year(s)",SLES for SAP Applications 2 Socket 24x7 SUSE S...,Computers & Electronics>Software>Software Lice...,Lenovo 00D8199 software license/upgrade SUSE L...,lenovo software suse linux enterprise server s...,"[sap, linux, lenovo, unix, server, enterprise,..."


In [3]:
df_validate = pd.read_json("icecat_preprocessed_validate_keybert.json")
df_validate.head()

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text,cleaned_text,keywords
0,PanzerGlass PG1501 screen protector Clear scre...,,PanzerGlass PG1501. Type: Clear screen protect...,"PanzerGlass PG1501, Clear screen protector, Mo...",Protective Anti Scratch Fluid Resistant Glass ...,Computers & Electronics>Telecom & Navigation>S...,PanzerGlass PG1501 screen protector Clear scre...,panzerglass screen protector clear screen prot...,"[panzerglass, protector, samsung, shield, glas..."
1,Verbatim 97537 keyboard Bluetooth QWERTY Black,"On the go, looking for a quick and easy way to...",Verbatim 97537. Connectivity technology: Wirel...,"Verbatim 97537, Wireless, Bluetooth, QWERTY, B...","Wireless, Bluetooth, iOS 4, f/iPad/iPhone/iPod...",Computers & Electronics>Data Input Devices>Key...,Verbatim 97537 keyboard Bluetooth QWERTY Black...,verbatim keyboard bluetooth qwerty black go lo...,"[keyboard, bluetooth, qwerty, wireless, iphone..."
2,"Symantec Backup Exec 2012 Agent f/ Windows, 1u...",<b>Symantec Backup Exec</b><br>\nReliable wind...,"Symantec Backup Exec 2012 Agent f/ Windows, 1u...","Symantec Backup Exec 2012 Agent f/ Windows, 1u...","Backup Exec 2012 Agent for Windows, 1 user, Ex...",Computers & Electronics>Software>Software Lice...,"Symantec Backup Exec 2012 Agent f/ Windows, 1u...",symantec backup exec agent window initial e rn...,"[symantec, backup, software, protects, license..."
3,"APC Change Mgr, 1 Year Software Maintenance Co...",Provides 24x7 telephone technical support and ...,"APC Change Mgr, 1 Year Software Maintenance Co...","APC Change Mgr, 1 Year Software Maintenance Co...","Change Mgr, 1 Year Software Maintenance Contra...",Computers & Electronics>Warranty & Support>War...,"APC Change Mgr, 1 Year Software Maintenance Co...",apc change mgr year software maintenance contr...,"[apc, maintenance, contract, software, change,..."
4,Urban Factory Adapter mini display port to HDM...,The Mini DisplayPort to HDMI adapter connects ...,Urban Factory Adapter mini display port to HDM...,Urban Factory Adapter mini display port to HDM...,Urban Factory Adapter mini display port to HDM...,Computers & Electronics>Computer Cables>Video ...,Urban Factory Adapter mini display port to HDM...,urban factory adapter mini display port hdmi m...,"[adapter, connector, displayport, hdmi, port, ..."


In [4]:
df_test = pd.read_json("icecat_preprocessed_test_keybert.json")
df_test.head()

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text,cleaned_text,keywords
0,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,In times of growing complexity and decreasing ...,Fujitsu FSP:G-SW3Z560PRE0S. Number of years: 3...,"Fujitsu FSP:G-SW3Z560PRE0S, 3 year(s), 9x5","SP 3y TS Sub & Upgr, 9x5, 4h Rm Rt f/ CS200c A...",Computers & Electronics>Warranty & Support>War...,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,fujitsu fsp extension time growing complexity ...,"[fujitsu, fsp, maintenance, availability, serv..."
1,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",<b>Revolutionary battery life on a convertible...,HP Spectre x360 13-ap0023tu. Product type: Hyb...,"HP Spectre x360 13-ap0023tu, 8th gen Intel® Co...","Intel® Core™ i7-8565U (1.8 GHz), 16GB DDR4-SDR...",Computers & Electronics>Computers>Notebooks,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",hp spectre blue silver hybrid cm x pixel touch...,"[hp, ssd, hybrid, spectre, portable, pixel, pr..."
2,HP OMEN 880-156nf 8th gen Intel® Core™ i7 i7-8...,,HP OMEN 880-156nf. Processor frequency: 3.2 GH...,"HP OMEN 880-156nf, 3.2 GHz, 8th gen Intel® Cor...",,Computers & Electronics>Computers>PCs/Workstat...,HP OMEN 880-156nf 8th gen Intel® Core™ i7 i7-8...,hp omen gen gb gb black tower pc hp omen proce...,"[hp, processor, memory, mhz, pc, gtx, ghz, sto..."
3,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,Perfect for your home office or a large instal...,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,Cat5E Assembled UTP Patch Cable Green 1m,Computers & Electronics>Computer Cables>Networ...,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,unshielded utp network patch cable green perfe...,"[cable, ethernet, network, gigabit, connector,..."
4,Lenovo 5PS0A14091 warranty/support extension,Lenovo offers a comprehensive portfolio of val...,"Lenovo 5PS0A14091. Number of years: 3 year(s),...","Lenovo 5PS0A14091, 3 year(s), On-site, 24x7, N...",3YR Onsite + Keep Your Drive,Computers & Electronics>Warranty & Support>War...,Lenovo 5PS0A14091 warranty/support extension L...,lenovo extension lenovo offer comprehensive po...,"[lenovo, service, extension, provides, provide..."


In [5]:
import pandas as pd
from tqdm import tqdm
import yake

In [6]:
# Initialize YAKE
yake_extractor = yake.KeywordExtractor(lan="en", n=1, top=10)

In [7]:
# Function to extract top 10 keywords using YAKE
def extract_keywords_yake(text):
    try:
        keywords = yake_extractor.extract_keywords(text)
        return [kw for kw, score in keywords]
    except:
        return []

In [8]:
# Function to merge existing KeyBERT keywords with YAKE keywords
def merge_keywords(row):
    kb = set(row.get("keywords", []))
    yk = set(row.get("yake_keywords", []))
    return list(kb.union(yk))

In [9]:
# Apply to each dataset
tqdm.pandas()

for df in [df_train, df_validate, df_test]:
    df["yake_keywords"] = df["cleaned_text"].progress_apply(extract_keywords_yake)
    df["combined_keywords"] = df.progress_apply(merge_keywords, axis=1)

100%|██████████| 90598/90598 [00:01<00:00, 56392.78it/s] 


In [11]:
df_train.head()

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text,cleaned_text,keywords,yake_keywords,combined_keywords
0,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,<b>Smart Multimedia Performance</b><br>\nVivoP...,ASUS K31CD-IT049T. Processor frequency: 3.4 GH...,"ASUS K31CD-IT049T, 3.4 GHz, 6th gen Intel® Cor...","Intel Core i7-6700 (8M Cache, 3.4GHz), 16GB RA...",Computers & Electronics>Computers>PCs/Workstat...,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,asus pc gen gb gb hdd black tower b smart mult...,"[hardware, audio, vivopc, asus, streaming, ram...","[asus, window, desktop, vivopc, faster, home, ...","[hdmi, hdtv, asus, streaming, graphic, faster,..."
1,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,Get the performance you demand at a price that...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,1m ST/SC Plenum-Rated 9/125 Duplex Single-Mode...,Computers & Electronics>Computer Cables>Fibre ...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,duplex fiber patch cable blue fiber optic cabl...,"[ethernet, fiber, cable, gigabit, duplex, tran...","[cable, connector, fiber, patch, blue, duplex,...","[bandwidth, patch, blue, cord, higher, gigabit..."
2,HP FA889AA Battery,Keeping an extra source of power nearby means ...,"HP FA889AA. Product type: Battery, Product col...","HP FA889AA, Battery, White, Lithium-Ion (Li-Io...","1100 mAh, Lithium Ion, Standard Battery",Computers & Electronics>Computers>Handheld Mob...,HP FA889AA Battery Keeping an extra source of ...,hp battery keeping extra source power nearby m...,"[hp, battery, power, ipaq, office, standard, l...","[battery, product, white, mah, keeping, extra,...","[nearby, product, messenger, power, office, ba..."
3,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,The C30 builds on its award-winning design as ...,Lenovo ThinkStation C30. Processor frequency: ...,"Lenovo ThinkStation C30, 2 GHz, Intel® Xeon® E...","Intel Xeon E5-2620 (15M Cache, 2.00 GHz, 7.20 ...",Computers & Electronics>Computers>PCs/Workstat...,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,lenovo thinkstation family gb gb hdd black tow...,"[lenovo, hardware, workstation, intel, process...","[workstation, thinkstation, memory, profession...","[chassis, design, technologyy, reliability, in..."
4,Lenovo 00D8199 software license/upgrade,SUSE Linux Enterprise Server (SLES)<br>\n<br>\...,Lenovo 00D8199. License term in years: 3 year(s),"Lenovo 00D8199, 3 year(s)",SLES for SAP Applications 2 Socket 24x7 SUSE S...,Computers & Electronics>Software>Software Lice...,Lenovo 00D8199 software license/upgrade SUSE L...,lenovo software suse linux enterprise server s...,"[sap, linux, lenovo, unix, server, enterprise,...","[enterprise, linux, server, suse, application,...","[sap, application, platform, year, sle, suppor..."


In [12]:
import pandas as pd

# Set display options to show full content
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

# Display the DataFrame
print(df_train[["keywords","yake_keywords","combined_keywords"]])


                                                                                                               keywords                                                                                    yake_keywords                                                                                                                                                 combined_keywords
0                                    [hardware, audio, vivopc, asus, streaming, ram, multimedia, processor, hdmi, hdtv]                     [asus, window, desktop, vivopc, faster, home, graphic, time, usb, processor]                             [hdmi, hdtv, asus, streaming, graphic, faster, vivopc, multimedia, audio, time, processor, home, desktop, window, hardware, usb, ram]
1                          [ethernet, fiber, cable, gigabit, duplex, transmission, connector, cord, network, bandwidth]                  [cable, connector, fiber, patch, blue, duplex, male, bandwidth, higher, jacket]                          

In [13]:
# Save updated datasets to JSON files
df_train.to_json("icecat_preprocessed_train_keybert_yake.json")
df_validate.to_json("icecat_preprocessed_validate_keybert_yake.json")
df_test.to_json("icecat_preprocessed_test_keybert_yake.json")